

下面的代码展示了Eisner算法：

In [2]:
# 代码来源于GitHub项目yzhangcs/crfpar 
# (Copyright (c) 2020 Yu Zhang, MIT License（见附录）)
import torch
import sys
sys.path.append('../code')
from utils import stripe, pad
def eisner(scores, mask):
    '''
    scores：大小为批大小 * 序列长度 * 序列长度，
    每个位置表示依存关系的打分，
    例如scores[0,1,2]就表示第0个输入样例上，
    边2->1的打分，2为中心词，1为依存词。
    
    mask：批大小 * 序列长度，掩码长度与句子长度相同。
    '''
    # 获取输入的基本信息
    lens = mask.sum(1)-1
    batch_size, seq_len, _ = scores.shape
    # 将scores矩阵从(batch,dep,head)形式转成(head,dep,batch)形式，
    # 方便并行计算
    scores = scores.permute(2, 1, 0)
    # 初始化不完整跨度情况下的打分
    s_i = torch.full_like(scores, float('-inf'))
    # 初始化完整跨度情况下的打分
    s_c = torch.full_like(scores, float('-inf'))
    # 保存两种情况下的max j的位置
    p_i = scores.new_zeros(seq_len, seq_len, batch_size).long()
    p_c = scores.new_zeros(seq_len, seq_len, batch_size).long()
    # 初始化完整跨度下长度为0的打分
    s_c.diagonal().fill_(0)

    for w in range(1, seq_len):
        # 通过seq_len - w可以计算出当前长度有多少长度为w的跨度
        n = seq_len - w
        # 根据n生成0到n的列表
        starts = p_i.new_tensor(range(n)).unsqueeze(0)
        
        # ---计算不完整跨度s(i,k,R,I)和s(i,k,L,I)的得分与最大值---
        
        # 计算s(i,j,R,C)+s(j+1,k,L,C)的值，
        # 对于s(i,k,R,I)和s(i,k,L,I)的计算过程中，这部分相同
        ilr = stripe(s_c, n, w) + stripe(s_c, n, w, (w, 1))
        # n * w * batch_size -> batch_size * n * w
        il = ir = ilr.permute(2, 0, 1)
        # 在s(i,k,L,I)中，计算max(s(i,j,R,C)+s(j+1,k,L,C))的值
        # 以及相应的位置
        il_span, il_path = il.max(-1)
        # 在求s_{ki}的过程时，我们的计算过程与第10章成分句法分析
        # 中的基于跨度的方法类似。
        # 不同的是由于k>i，因此在diagonal命令时需要用-w，让对角线下移
        # 具体细节请查看PyTorch文档
        s_i.diagonal(-w).copy_(il_span + scores.diagonal(-w))
        # 保留最大的j值
        p_i.diagonal(-w).copy_(il_path + starts)
        
        # 在s(i,k,R,I)中，计算max(s(i,j,R,C)+s(j+1,k,L,C))的值
        # 以及相应的位置
        ir_span, ir_path = ir.max(-1)
        # 求s_{ik}，此时对角线上移
        # 与此同时，这种方式可以保证s_i保存的方向为L的值与
        # 方向为R的值互相不冲突，下同
        s_i.diagonal(w).copy_(ir_span + scores.diagonal(w))
        # 保留最大的j值
        p_i.diagonal(w).copy_(ir_path + starts)
        
        
        # ---计算不完整跨度s(i,k,R,C)和s(i,k,L,I)的得分与最大值---
        
        # 计算 s(i,j,L,C)+s(j,k,L,I) 
        cl = stripe(s_c, n, w, (0, 0), 0) + stripe(s_i, n, w, (w, 0))
        cl_span, cl_path = cl.permute(2, 0, 1).max(-1)
        # 将最大的得分进行保存
        s_c.diagonal(-w).copy_(cl_span)
        # 将最大的得分的位置进行保存
        p_c.diagonal(-w).copy_(cl_path + starts)
        
        # 计算 s(i,j,R,I)+s(j,k,R,C)
        cr = stripe(s_i, n, w, (0, 1)) + stripe(s_c, n, w, (1, w), 0)
        cr_span, cr_path = cr.permute(2, 0, 1).max(-1)
        # 将最大的得分进行保存
        s_c.diagonal(w).copy_(cr_span)
        # 将句子长度不等于w的(0,w)得分置为负无穷，
        # 因为其在结构上不可能存在
        s_c[0, w][lens.ne(w)] = float('-inf')
        # 将最大的得分的位置进行保存
        p_c.diagonal(w).copy_(cr_path + starts + 1)

    def backtrack(p_i, p_c, heads, i, k, complete):
        # 通过分治法找到当前跨度的最优分割
        if i == k:
            return
        if complete:
            # 如果当前跨度是完整跨度，取出得分最大的位置
            j = p_c[i, k]
            # 分别追溯s(i,j,I)和s(j,k,C)的最大值
            backtrack(p_i, p_c, heads, i, j, False)
            backtrack(p_i, p_c, heads, j, k, True)
        else:
            # 由于当前跨度是不完整跨度，因此根据定义，k的父节点一定是i
            j, heads[k] = p_i[i, k], i
            i, k = sorted((i, k))
            # 追溯s(i,j,C)和s(j+1,k,C)的最大值
            backtrack(p_i, p_c, heads, i, j, True)
            backtrack(p_i, p_c, heads, k, j + 1, True)

    preds = []
    p_c = p_c.permute(2, 0, 1).cpu()
    p_i = p_i.permute(2, 0, 1).cpu()
    # 追溯最终生成的每个词的父节点
    for i, length in enumerate(lens.tolist()):
        heads = p_c.new_zeros(length + 1, dtype=torch.long)
        backtrack(p_i[i], p_c[i], heads, 0, length, True)
        preds.append(heads.to(mask.device))

    return pad(preds, total_length=seq_len).to(mask.device)


给定输入句子“she learns the book hands-on-NLP”，<!--我们的依存分析模型为它的每个部分的打分，-->让我们来看最终输出结果：

In [24]:
score = torch.Tensor([
    [ -1,  -1,  -1,  -1,  -1, -1],
    [ -1,  -1,  1,  -1,  -1, -1],
    [ 1, -1, -1, -1, -1, -1],
    [ -1, -1, -1, -1, -1, 1],
    [ -1, -1, -1, -1, -1, 1],
    [ -1, -1, 1, -1, -1, -1]]).unsqueeze(0)

mask = torch.ones_like(score[:,:,0]).long()

deps=eisner(score,mask)
# deps 中第0位为根节点
print(deps)

tensor([[0, 2, 0, 5, 5, 2]])


现在，我们来画一下这个依存句法树。这里使用HanLP代码包来画这个依存句法树。<!--hanlp是一个优秀的面向中文以及其他多语言的自然语言处理工具包。-->由于没有为标签进行打分，因此这里只给根节点打上ROOT标签，其余依存边无标签。

In [20]:
# !pip install -e hanlp_common
from hanlp_common.document import Document
# from document import Document

tokens = ["she","learns","the","book","hands-on-NLP"]
dependencies = [[x.item(), '' if x.item()!=0 else "ROOT"]\
    for x in deps[0,1:]]
doc = Document(tok=tokens,dep=dependencies)
doc.pretty_print()

以下是MST的代码实现。

In [38]:
# 代码来源于GitHub项目tdozat/Parser-v1 
# (Copyright (c) 2016 Timothy Dozat, Apache-2.0 License（见附录）)
import numpy as np
from tarjan import Tarjan

def MST_inference(parse_probs, length, mask, ensure_tree = True):
    # parse_probs：模型预测的每个词的父节点的概率分布，
    # 大小为 length * length，顺序为(孩子节点,父节点)
    # length：当前句子长度
    # mask：与parse_probs大小一致，表示这句话的掩码
    if ensure_tree:
        # 根据mask大小，生成单位矩阵
        I = np.eye(len(mask))
        # 去除不合理元素，其中，通过(1-I)将对角线上的元素去除，
        # 因为句法树不可能存在自环
        parse_probs = parse_probs * mask * (1-I)
        # 求出每个位置上概率最高的父节点
        parse_preds = np.argmax(parse_probs, axis=1)
        tokens = np.arange(1, length)
        # 确认目前的根节点
        roots = np.where(parse_preds[tokens] == 0)[0]+1
        # 当没有根节点时，保证至少有一个根节点
        if len(roots) < 1:
            # 当前每个位置对根节点的概率
            root_probs = parse_probs[tokens,0]
            # 当前每个位置对概率最高的节点的概率
            old_head_probs = parse_probs[tokens, parse_preds[tokens]]
            # 计算根节点与概率最高节点的比值，作为选取根节点的相对概率
            new_root_probs = root_probs / old_head_probs
            # 选择最可能的根节点
            new_root = tokens[np.argmax(new_root_probs)]
            # 更新预测结果
            parse_preds[new_root] = 0
        # 当根节点数量超过1时，让根节点数量变为1
        elif len(roots) > 1:
            # 当前父节点的概率
            root_probs = parse_probs[roots,0]
            # 让当前所有的依存于根节点的位置（roots）归零
            parse_probs[roots,0] = 0
            # 获得新的潜在的父节点及其概率
            new_heads = np.argmax(parse_probs[roots][:,\
                tokens], axis=1)+1
            new_head_probs = parse_probs[roots,\
                new_heads] / root_probs
            # 选择roots的潜在的新的父节点中，概率最小的位置，
            # 将其父节点作为根节点
            new_root = roots[np.argmin(new_head_probs)]
            # 更新预测结果
            parse_preds[roots] = new_heads
            parse_preds[new_root] = 0
        # 在通过贪心的方式获得所有位置的父节点后，
        # 使用Tarjan算法找到当前图中的强联通分量，
        # 使用MST算法将其中的环接触并且重新进行链接
        tarjan = Tarjan(parse_preds, tokens)
        # 当前的强联通分量（环）
        cycles = tarjan.SCCs
        for SCC in tarjan.SCCs:
            # 当强联通分量里的节点数量超过1个，那么说明其有环
            if len(SCC) > 1:
                dependents = set()
                to_visit = set(SCC)
                # 将环内所有的节点以及它们所连接的外部节点
                # 都加入孩子节点中
                while len(to_visit) > 0:
                    node = to_visit.pop()
                    if not node in dependents:
                        dependents.add(node)
                        # 将当前节点指向的节点（孩子节点）
                        # 加入要访问的队列中
                        to_visit.update(tarjan.edges[node])
                # 参与循环的节点的位置
                cycle = np.array(list(SCC))
                # 当前父节点的概率
                old_heads = parse_preds[cycle]
                old_head_probs = parse_probs[cycle, old_heads]
                # 为了计算环里每个节点的新的父节点，
                # 这些节点的孩子节点是这些节点的父节点显然是不可能的，
                # 因此需要将它们的概率置为0
                non_heads = np.array(list(dependents))
                parse_probs[np.repeat(cycle, len(non_heads)),\
                    np.repeat([non_heads], len(cycle),\
                    axis=0).flatten()] = 0
                # 新的概率分布下，求得环内所有节点新的
                # 潜在父节点及其概率
                new_heads = np.argmax(parse_probs[cycle][:,\
                    tokens], axis=1)+1
                # 与旧的父节点计算比例
                new_head_probs = parse_probs[cycle,\
                    new_heads] / old_head_probs
                # 选择最有可能的变化，这样对于树的整体概率
                # 影响最小，同时能将当前的环解除
                change = np.argmax(new_head_probs)
                changed_cycle = cycle[change]
                old_head = old_heads[change]
                new_head = new_heads[change]
                # 更新预测结果
                parse_preds[changed_cycle] = new_head
                tarjan.edges[new_head].add(changed_cycle)
                tarjan.edges[old_head].remove(changed_cycle)
        return parse_preds
    else:
        # 当不强制要求树结构时，直接将预测结果返回
        parse_probs = parse_probs * mask
        parse_preds = np.argmax(parse_probs, axis=1)
        return parse_preds

下面，我们设计一个使用11.2.2节所介绍的中心词选择解码会导致有环的情况，来看看MST算法的运行结果：

In [42]:

# 第5个词的分数最高的中心词为第4个词，形成4->5 5->4的环
score = np.array([
    [ -1,  -1,  -1,  -1,  -1, -1],
    [ -1,  -1,  1,  -1,  -1, -1],
    [ 1, -1, -1, -1, -1, -1],
    [ -1, -1, -1, -1, -1, 1],
    [ -1, -1, -1, -1, -1, 1],
    [ -1, -1, 1, -1, 1.1, -1]]) 

mask = np.ones_like(score)
# 可以看出直接预测最大值会有环形成
print('不使用MST算法得到的依存关系为：',np.argmax(score,1))
deps=MST_inference(score,len(mask),mask)
print('使用MST算法得到的依存关系为：',deps)

不使用MST算法得到的依存关系为： [0 2 0 5 5 4]
使用MST算法得到的依存关系为： [0 2 0 5 5 2]


这里我们来简单求一下边的交叉熵损失：


In [23]:
score = torch.Tensor([
    [ -1,  -1,  -1,  -1,  -1, -1],
    [ -1,  -1,  1,  -1,  -1, -1],
    [ 1, -1, -1, -1, -1, -1],
    [ -1, -1, -1, -1, -1, 1],
    [ -1, -1, -1, -1, -1, 1],
    [ -1, -1, 1, -1, 1.1, -1]])
# 假设我们的目标
target = torch.Tensor([2,0,5,5,2]).long()
# 计算交叉熵损失
loss_func = torch.nn.NLLLoss()
loss = loss_func(torch.nn.functional.log_softmax(score[1:],-1),target)
print(loss)

tensor(0.6081)


<!--#### 推理代码实现-->

下面提供一套代码来展示在解码过程中如何根据转移动作的打分去对栈和缓存进行操作。如果读者想要运行该代码，需自行定义model。

In [1]:
SHIFT_ID=0
# 假设left_arc有两个label，nsubj和dobj
LEFT_ARC_ID = {1: 'nsubj',2: 'dobj'}
# 假设right_arc有3个label，nsubj、dobj和root
RIGHT_ARC_ID = {3:'nsubj',4:'dobj',5:'root'}


def decode(words,model):
    # words：每个元素为(word_idx, word_text)的元组，
    # word_idx为句子中的位置，word_text则为文本
    # model：这里不具体构建模型，仅作为一个示例
    # 缓存buffer初始化，将words翻转，能够保证pop()操作
    # 能够从前往后进行
    buffer = words[::-1]
    # 栈stack初始化，0表示root节点
    stack = [(0,'ROOT')]
    # 保存生成的边
    deps = []
    # 循环转移迭代
    while 1:
        # 模型通过buffer、stack和history计算下一步操作的打分
        log_probs = model(buffer,stack,history)
        # 得到得分最高的操作id，范围为[0,5]
        action_id = torch.max(log_probs)[1]
        # 当action_id分别为0、1和大于1时，分别为其做SHIFT、
        # REDUCE和push_nt操作
        if action_id == SHIFT_ID:
            buffer,stack = shift(buffer,stack)
        elif action_id in LEFT_ARC_ID:
            stack,deps = left_arc(stack,deps,action_id)
        else:
            stack,deps = right_arc(stack,deps,action_id)
        
        # 当缓存为空，栈只有一个子树时则退出
        if len(buffer) == 0 and len(stack) == 1:
            break
    # 返回生成的树
    return deps

def shift(buffer,stack):
    # 将buffer中的词移动到栈顶
    word=buffer.pop()
    # 这里只需要保留word_idx
    stack.append(word)
    return buffer, stack 

def left_arc(stack,deps,action_id):
    # 因为是向左的弧，所以取出stack最后的两个词，倒数第一个词为中心词
    head_word = stack.pop()
    dep_word = stack.pop()
    # 保存head，dep位置以及它们所对应的边，只需要保存word_idx
    deps.append((head_word[0],dep_word[0],LEFT_ARC_ID[action_id]))
    # 将中心词放回stack中
    stack.append(head_word)
    return stack, deps

def right_arc(stack,deps,action_id):
    # 因为是向右的弧，所以取出stack最后的两个词，倒数第二个词为中心词
    dep_word = stack.pop()
    head_word = stack.pop()
    # 保存head，dep位置以及它们所对应的边
    deps.append((head_word[0],dep_word[0],LEFT_ARC_ID[action_id]))
    # 将中心词放回stack中
    stack.append(head_word)
    return stack, deps